# Similarity

In [ ]:
# Librairies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import random
import time

from Preprocessing import *

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split, cross_val_score

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 60)

## Load data

In [2]:
# Data labellisées (catégories Innoscape)
df = pd.read_csv('../Data/t3_all_fr_prices_posweekly_prod_without_price.csv')

# Colonnes à garder
col = ['t3_all_fr_prices_posweekly_prod.productfamily_seller',
       't3_all_fr_prices_posweekly_prod.prodcategory4_seller',
       't3_all_fr_prices_posweekly_prod.prodcategory3_seller',
       't3_all_fr_prices_posweekly_prod.prodcategory2_seller',
       't3_all_fr_prices_posweekly_prod.prodcategory1_seller',
       't3_all_fr_prices_posweekly_prod.productseller_name',
       't3_all_fr_prices_posweekly_prod.id_source',
       't3_all_fr_prices_posweekly_prod.brand_seller']

df = df[col]

# Raccourcir nom des colonnes
raccourci_col = map(lambda string: string.replace('t3_all_fr_prices_posweekly_prod.',''), col)
col_name_dict = dict(zip(col, raccourci_col))

# Renommer nom des colonnes
df.rename(columns=col_name_dict, inplace=True)


In [3]:
# Exemples interessants : plusieurs mesures
print('des :', df['productseller_name'].iloc[26])
print('des :', df['productseller_name'].iloc[78])
print('des :', df['productseller_name'].iloc[30843])


des : Ryobi-RXPR01 Podadora-chaîne 254 mm-larg. 150 cm DIAMETRE-Coupe 20 cm
des : Nortene 172587 0,8 mm x 50 m Fil Plastifié Support utilisé pour filets – Vert
des : MASTIC COLLE POUR MUR ET FACADE SIKAFLEX 310ML TON PIERRE


In [4]:
test = 'colle maxtack tuile 300 ml'
x = re.search("([\d.]+)\s*(\bm\b|ml)", test)

if x is not None:
    print(x)
else:
    print('Non')

<_sre.SRE_Match object; span=(20, 26), match='300 ml'>


## Preprocessing

In [5]:
# Lettre en minuscule
def str_lower(df):
    for col in df.columns:
        df[col] = df[col].str.lower()
    return df

df = str_lower(df)

In [6]:
# Récupérer les attributs : Poids, Mesure, Volume, Couleur
def preprocessing_attributs(df, name_col):
    df['Poids'] = df[name_col].apply(lambda x: re.search("([\d.,]+)\s*(g|kg)", x).group() if re.search("([\d.,]+)\s*(g|kg)", x) is not None else '')
    df['Mesure'] = df[name_col].apply(lambda x: re.search("([\d.,]+)\s*(mm|cm|\bm\b)", x).group() if re.search("([\d.,]+)\s*(mm|cm|\bm\b)", x) is not None else '')
    df['Volume'] = df[name_col].apply(lambda x: re.search("([\d.,]+)\s*(ml|cl|l)", x).group() if re.search("([\d.,]+)\s*(ml|cl|l)", x) is not None else '')
    df['Couleur'] = df[name_col].apply(lambda x: re.search("(gris|noir|rouge|blanc|bleu|vert|argent|multicolore)", x).group() if re.search("(gris|noir|rouge|blanc|bleu|vert|argent|multicolore)", x) is not None else '')
    return df

df = preprocessing_attributs(df, 'productseller_name')


In [7]:
# Normaliser unité de mesure 
# Exemple : poids en gramme, mesure en mètre, volume en litre
def normalization_unit(df, name_col):
    
    dico = {'Poids':{'g':1, 'kg':1000}, 'Mesure':{'mm':0.001, 'm':1, 'cm':0.01}, 'Volume':{'ml':0.001, 'cl':0.01, 'l':1}}
    dico_regex = {'Poids':'(g|kg)', 'Mesure':'(mm|cm|m)', 'Volume':'(ml|cl|l)'}
    
    df[name_col + '_unit'] = df[name_col].apply(lambda x: float(re.search('\d+', x).group())*dico[name_col][re.search(dico_regex[name_col], x).group()] if re.search("\d+", x) and re.search(dico_regex[name_col], x) is not None else '')
    
    return df

df = normalization_unit(df, 'Poids')
df = normalization_unit(df, 'Mesure')
df = normalization_unit(df, 'Volume')

In [8]:
df.head(200)

productfamily_seller  \
0                       colles à carrelage   
1                                      NaN   
2                                      NaN   
3                                   mastic   
4                                    cires   
5                                      NaN   
6                        perceuse-visseuse   
7                    produits d'étanchéité   
8                                arroseurs   
9                                      NaN   
10                                     NaN   
11                   pompes pour fontaines   
12                    sécateurs et ciseaux   
13     accessoires pour outillage sans fil   
14   pièces et accessoires pour chaudières   
15                                     NaN   
16      filtres pour aspirateurs portatifs   
17                       meuleuses d'angle   
18                                     NaN   
19                                conduits   
20                                conduits   
21                         mastic au latex   
22                      pièces et raccords   
23                    casques et écouteurs   
24                    bordures pour jardin   
25                     marteau perforateur   
26                           tronçonneuses   
27                         stickers muraux   
28                      colles à carrelage   
29                                     NaN   
..                                     ...   
170                                    NaN   
171                                    NaN   
172                       bêches et pelles   
173                     climatiseurs split   
174                                    NaN   
175                                    NaN   
176                      packs de batterie   
177                                    NaN   
178                                 mastic   
179                              silicones   
180                     filets pour jardin   
181              rasoirs à têtes rotatives   
182                        perceuses combi   
183                              arroseurs   
184                                    NaN   
185                     filets pour jardin   
186                                  cires   
187                                    NaN   
188                                    NaN   
189                                    NaN   
190                                 forets   
191                                    NaN   
192                                    NaN   
193                                    NaN   
194                      packs de batterie   
195                                    NaN   
196                                    NaN   
197                         transplantoirs   
198                                  cires   
199                                    NaN   

                      prodcategory4_seller  \
0                       colles à carrelage   
1                                      NaN   
2                                      NaN   
3                                   mastic   
4                                    cires   
5                                      NaN   
6                                perceuses   
7                    produits d'étanchéité   
8                                arroseurs   
9                                      NaN   
10                                     NaN   
11                                  pompes   
12                    sécateurs et ciseaux   
13     accessoires pour outillage sans fil   
14   pièces et accessoires pour chaudières   
15                                     NaN   
16                                 filtres   
17                               meuleuses   
18                                     NaN   
19                                conduits   
20                                conduits   
21                                  mastic   
22                   tuyaux et accessoires   
23                    casques et écouteurs   
24                 

In [152]:
# Test
dico = {'Poids':{'g':1, 'kg':1000}, 'Mesure':{'mm':0.001, 'm':1, 'cm':0.01}, 'Volume':{'ml':0.001, 'cl':0.01, 'l':1}}

test = '25g'
if re.search('\d+', test) and re.search('(g|kg)', test) is not None:
    print(float(re.search('\d+', test).group())*dico['Poids'][re.search('(g|kg)', test).group()])
else:
    print('non')


25.0
